# Load Layer outputs

In [1]:
nn = 'NN'

import pickle
with open('results/' + nn + '_layer_outputs.dat','rb') as f:
    layer_outs,layer_outs_test= pickle.load(f)
f.close()

# Load recording and test data

In [ ]:
import numpy as np
from keras.datasets import mnist
from scipy.misc import imresize

(Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()

(ntrain, xdim, ydim) = Xtrain.shape
ntest = Xtest.shape[0]

# Recording data
X_pr = Xtrain[30000:60000, :, :]
Y_pr = Ytrain[30000:60000]


# downsample
factor = 1
if factor<1:
    Xtest_down = np.ones((Xtest.shape[0], int(xdim*factor), int(ydim*factor)))
    for i in range(Xtest.shape[0]):
        Xtest_down[i,:,:] = imresize(Xtest[i,:,:], factor)

    X_pr_down = np.ones((X_pr.shape[0], int(xdim*factor), int(ydim*factor)))
    for i in range(X_pr.shape[0]):
        X_pr_down[i,:,:] = imresize(X_pr[i,:,:], factor)
else:
    Xtest_down = Xtest
    X_pr_down = X_pr
    
# VECTORIZE IMAGES
Xtest_down = Xtest_down.reshape(ntest, int(xdim*factor)**2).astype('float32') / 255
X_pr_down = X_pr_down.reshape(X_pr_down.shape[0], int(xdim*factor)**2).astype('float32') / 255

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: GeForce GTX TITAN X (CNMeM is enabled with initial size: 50.0% of memory, cuDNN 5103)


# Run XGBOOST

In [ ]:
from copy import copy, deepcopy
from RE_PartialRecData import RE_PartialRecData
from RE_PartialRecData2 import RE_PartialRecData2
import os
import xgboost as xgb
import sklearn
from xgboost.sklearn import XGBClassifier
from fancyimpute import SoftImpute as FI #SimpleFill, KNN, NuclearNormMinimization, SoftImpute
import pickle


params = {}
# use softmax multi-class classification 'multi:softmax'
# use linear regression 'reg:linear'
params['objective'] = 'reg:linear'
# scale weight of positive examples
params['eta'] = 0.5               # Makes the model more robust by shrinking the weights on each step (0.01-0.2)
params['max_depth'] = 6           # Used to control over-fitting as higher depth will allow model to learn relations 
                                  # very specific to a particular sample. (3-10)
params['silent'] = 1
params['nthread'] = 4
# params['num_class'] = 10
num_round = 5

# how many recordings?
recordings = [10]
nRecordings = len(recordings)
# which layers?
layers = [0,2]
# how many iterations
nIterations = 5

# baseline prediction error
#bl = np.std(layer_outs_test[oLayer]-np.mean(layer_outs_test[oLayer]));

oLayer = len(layer_outs)-1  # index of output layer
nOutNeurons = layer_outs[oLayer].shape[1]

for iLayer in layers:
    # how many neurons from the firs hidden layer?
    subnetSize = [2**x for x in range(np.int(np.log2(layer_outs[iLayer].shape[1])+1))]
    nSubnetSize = len(subnetSize)
    # how many samples per recording?
    nSamples = np.divide(int(X_pr_down.shape[0]/nRecordings),subnetSize)*100
    
    rmses = np.zeros([nOutNeurons, nIterations, nSubnetSize, nRecordings])
    for nr in range(nRecordings):
        # how many samples per recording?
        nSamples = np.divide(int(X_pr_down.shape[0]/recordings[nr]),subnetSize)*100
        for ss in range(nSubnetSize):
            print(subnetSize[ss])
            for it in range(nIterations):
                # copy data
                layer_outputs = deepcopy(layer_outs)
                # subsample
                X_subsample, Y_subsample = RE_PartialRecData2(layer_outputs[iLayer], layer_outputs[oLayer], 
                                                              subnetSize[ss], recordings[nr],nSamples[ss])
                if ss==nSubnetSize-1:
                    X_impute = X_subsample
                else:
                    X_impute = FI(convergence_threshold=0.01, max_iters=30).complete(X_subsample)

        #        print('# nan values: ',np.count_nonzero(np.isnan(X_subsample)))
                # prepare data for xgboost
                for iN in range(nOutNeurons):
                    print('#neuron, #iteratin, subnetsize: ', iN,it,subnetSize[ss])
                    xg_train  = xgb.DMatrix(X_impute, label=Y_subsample[:, iN])
                    xg_test   = xgb.DMatrix(layer_outs_test[iLayer], label=layer_outs_test[oLayer][:,iN])
                    watchlist = [(xg_train, 'train'), (xg_test, 'test')]
                    # train XGboost
                    bst = xgb.train(params, xg_train, num_round, watchlist, verbose_eval=False)
                    # get predictions
                    pred = bst.predict(xg_test)
                    rmses[iN,it,ss,nr] = np.sqrt(np.mean(np.square([(pred[i] - layer_outs_test[oLayer][:,iN][i]) 
                                                 for i in range(len(layer_outs_test[oLayer][:,1]))])))
                print ('predicting, RMSE=%f' %np.mean(rmses[:, it, ss, nr]))

    # save the rmse's
    fName = 'results/softImpute_XGB_RMSES_Layer' + str(iLayer) + '_nRec' + str(nRecordings) + '_' + nn + '.dat'
    with open(fName,'wb') as f:
        pickle.dump(rmses, f)
    f.close()

1
[SoftImpute] Max Singular Value of X_init = 1432.998201
[SoftImpute] Iter 1: observed MAE=0.019742 rank=10


# Plot RMSE's vs. No. observed neurons per recordings

In [ ]:
# import matplotlib.pyplot as pl
# %matplotlib inline
# import numpy as np

# fig=pl.figure(figsize=(10,6))
# ax1 = fig.add_subplot(111)
# ax1.set_xlim([0, 128])
# ax2 = ax1.twiny()

# x = subnetSize;
# y = np.mean(np.median(rmses, axis=1), axis=0)
# error = np.std(np.mean(rmses, axis=1), axis=0)
# bl = np.std(layer_outs_test[oLayer]-np.mean(layer_outs_test[oLayer]));

# pl.semilogx(x, y, 'k-')
# # horiz_line_data = np.array([bl for i in xrange(len(x))])
# # pl.plot(x, horiz_line_data, 'k--') 
# pl.fill_between(x, y-error, y+error, alpha=0.2, facecolor='#808080')

# ax1.set_xlabel('# observed neurons on Layer' + str(iLayer) + '(out of 128)', fontsize=18)
# ax2.set_xlabel('Samples per recording',  fontsize=16)

# new_tick_locations = subnetSize
# ax1.set_xlim(ax1.get_xlim())
# ax1.set_xticks(new_tick_locations)
# ax1.set_xticklabels(new_tick_locations)

# new_tick_locations = subnetSize
# ax2.set_xlim(ax1.get_xlim())
# ax2.set_xticks(new_tick_locations)
# ax2.set_xticklabels(nSamples)


# # ax2.set_xticks(nSamples)
# # ax2.set_xticklabels(nSamples[range(0, 5, 20)])
# ax1.set_ylabel('RMSE', fontsize=18)
# ax1.set_ylim([0, .50])

# # pl.text(110,bl+0.005, 'baseline')
